### selenium을 이용해 데이터셋 구축을 위한 크롤링
* pip install selenium
* pip install webdriver_manager

In [10]:
import time
import urllib.request

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
# Mac에서 실행할 때

#from selenium import webdriver
#from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

### 인스타그램에서 이미지 크롤링
* 인스타그램은 인터페이스가 자주 바뀌기 때문에 크롤링을 위해선 코드를 주기적으로 최신화해줘야 한다.

In [ ]:
# 드라이버 설정 및 인스타그램 로그인
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://www.instagram.com/"
driver.get(url)

wait = WebDriverWait(driver, 10)


# 로그인 과정
input_id = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[aria-label="전화번호, 사용자 이름 또는 이메일"]')))
input_id.clear()
input_id.send_keys('인스타그램아이디')

input_pw = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[aria-label="비밀번호"]')))
input_pw.clear()
input_pw.send_keys('비밀번호')

login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))
login_button.click()

# "나중에 하기" 버튼 클릭
try:
    not_now_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@role='button'][contains(text(), '나중에 하기')]")))
    not_now_button.click()
except:
    print("나중에 하기 버튼 클릭 실패")

print("로그인 완료")

svg_xpath = '//*[@id="mount_0_0_br"]/div/div/div/div[2]/div/div/div[1]/div[1]/div[2]/div/div/div/div/div[2]/div[2]/span/div/a/div/div[1]/div/div/svg'


# 페이지가 로드되었는지 확인 (네비게이션 바가 로드될 때까지 대기)
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, "//div[@role='navigation']"))
)

time.sleep(3)

# 돋보기 모양의 SVG 아이콘 클릭
try : 
    search_button = wait.until(EC.element_to_be_clickable((By.XPATH, svg_xpath)))
    search_button.click()
except Exception as e : 
    print(f'svg 클릭실패{e}')
    


In [ ]:
search = "#증명사진" #검색할 키워드
time.sleep(2)

# 검색창 찾기 및 검색어 입력
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@placeholder='검색']"))
)
search_box.clear()
search_box.send_keys(search)
time.sleep(3)
print("검색 완료")

# 첫 번째 검색 결과 클릭
first_result = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{search}')]"))
)
first_result.click()

# 스크롤 내리기
SCROLL_PAUSE_TIME = 2 # 2초마다 스크롤 내리기
last_height = driver.execute_script("return document.body.scrollHeight")
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 이미지 찾기 및 다운로드
images = driver.find_elements(By.XPATH, "//img[@class='x5yr21d xu96u03 x10l6tqk x13vifvy x87ps6o xh8yej3']")
print(f"총 {len(images)}개 이미지")

for count, image in enumerate(images):
    try:
        imgUrl = image.get_attribute("src")
        print(f"{count+1}번째 이미지 저장 시작")
        urllib.request.urlretrieve(imgUrl, f"{count+1}.jpg")
        print(f"{count+1}번째 이미지 저장 완료")
        time.sleep(1)  # 다운로드 간 간격
    except Exception as e:
        print(f"이미지 저장 실패: {e}")